In [1]:
##We need to run these cells in order to get all the necessary imports

In [2]:
pip install requests

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyobb

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install Pillow==2.2.1

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from pydrake.systems.sensors import RgbdSensor, PixelType
from pydrake.multibody.plant import MultibodyPlant

from pydrake.math import RigidTransform, RotationMatrix, RollPitchYaw
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.perception import DepthImageToPointCloud, BaseField
from pydrake.systems.framework import DiagramBuilder, AbstractValue
from pydrake.geometry.render import (CameraProperties, DepthCameraProperties,
                                     MakeRenderEngineVtk, RenderEngineVtkParams)
from pydrake.systems.analysis import Simulator
from pydrake.systems.meshcat_visualizer import (MeshcatVisualizer,
    MeshcatPointCloudVisualizer, AddTriad)


In [7]:
# Construct a drake simulation diagram builder.
builder = DiagramBuilder()

# Construct empty MultibodyPlant.
plant = MultibodyPlant(2e-3)
_, scene_graph = AddMultibodyPlantSceneGraph(builder, plant=plant)
parser = Parser(plant=plant, scene_graph=scene_graph)
plant.mutable_gravity_field().set_gravity_vector([0, 0, 0])

# Add iiwa to MultibodyPlant
robot_sdf_path = FindResourceOrThrow(
    "drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
parser.AddModelFromFile(robot_sdf_path)
plant.WeldFrames(A=plant.world_frame(),
                 B=plant.GetFrameByName("iiwa_link_0"))

# Add a box to MultibodyPlant as the ground.
ground_sdf_path = os.path.join(
    os.getcwd(), "ground_box.sdf")
parser.AddModelFromFile(ground_sdf_path)

# Define pose of the box in world frame.
X_WB = RigidTransform.Identity()
X_WB.set_translation([0, -(2.5 - np.sqrt(2) - 0.1), -0.6])  # "ground"
plant.WeldFrames(A=plant.world_frame(),
                 B=plant.GetFrameByName("ground"),
                 X_AB=X_WB)

# Finalize MultibodyPlant.
plant.Finalize()

NameError: name 'os' is not defined

In [ ]:
# Add meshcat visualizer to diagram.
vis = builder.AddSystem(MeshcatVisualizer(
    scene_graph, zmq_url="tcp://127.0.0.1:6000"))

builder.Connect(scene_graph.GetOutputPort("lcm_visualization"),
                vis.get_input_port(0))
scene_pc_vis = builder.AddSystem(MeshcatPointCloudVisualizer(
    vis, name="scene_point_cloud"))
builder.Connect(di2pc.GetOutputPort("point_cloud"),
                scene_pc_vis.GetInputPort("point_cloud_P"))

# Complete diagram construction.
diagram = builder.Build()

# Render diagram structure in a PDF file (saved in the directory of this notebook).
RenderSystemWithGraphviz(diagram)

In [ ]:
##############################################################################

In [9]:
import requests
import numpy as np
import urllib


# http call for the mask 
def get_mask(url):
    response = requests.get(url)
    if response is not None:
        return np.frombuffer(response.content, dtype=np.uint8)

In [10]:
def make_meshcat_color_array(N, r, g, b):
    '''
    Construct a color array to visualize a point cloud in meshcat  Args:
    N: int. number of points to generate. Must be >= number of points in the
      point cloud to color
    r: float. The red value of the points, 0.0 <= r <= 1.0
    g: float. The green value of the points, 0.0 <= g <= 1.0
    b: float. The blue value of the points, 0.0 <= b <= 1.0  Returns:
    3xN numpy array of the same color
    '''
    color = np.zeros((3, N))
    color[0, :] = r
    color[1, :] = g
    color[2, :] = b  
    return color

In [11]:
# http call for the mask 
def get_box(url):
    response = requests.get(url)
    response = response.json()
    val = response['box']
    val = val.strip('][').split(',')
    val = [int(x) for x in val]
    if val is not None:
        return val

In [12]:
import numpy as np
import os
import time
import cv2
import os
import time
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf
from meshcat.animation import Animation
from math import pi, cos, sin, sqrt
from pyobb.obb import OBB
from PIL import Image
import matplotlib.pyplot as plt
import requests

vis = meshcat.Visualizer().open()

# url to the server
mask_url = "http://10.150.0.37:5000/mask" #mask url
box_url = "http://10.150.0.37:5000/box" #mask url

depth_url = 'http://10.150.0.37:5000/video_feed_dpt' #depth url

stream = urllib.request.urlopen(depth_url) #stream

# arrays for segmented and unsegmented points
bbox_pos_pts = []
bbox_clr_pts = []

time.sleep(1)
time_start = time.time()

# bytes for the pointcloud, keeps the last pieces as well.
bytes = b''

# we use a while loop in order to. continue listen to the port to get deth data
while True:  # show streamed images until Ctrl-C
    
    ############ MASKS MODIFICATION #############
    
    try:
        mask_im_obj = get_mask(mask_url)
        mask_im_arr = cv2.imdecode(mask_im_obj, flags=1)
        im_box = get_box(box_url)        
        
        
    except Exception as e:
        # sometimes we run into a buffer exception from cv2
        continue

    #rotate image data 90 degrees CCW
    mask_im_arr = np.rot90(mask_im_arr) 
    mask_im_arr = np.array(np.transpose(mask_im_arr, (1, 0, 2))[:, ::-1])
    mask_im_arr = cv2.resize(mask_im_arr, (320,240))
    mask_im_arr = mask_im_arr[:,:,0]
    
    x1,y1,x2,y2 = [int(i/2) for i in im_box]
    mask_im_arr = mask_im_arr[y1:y2,x1:x2]
    
    sz_a, sz_b = list(np.shape(mask_im_arr))
            
    pix_res = sz_a * sz_b 
    pix = np.reshape(mask_im_arr, (pix_res,1))
    
    ############ POINTCLOUD BYTES #############

    bbox_pos_pts = []
    bbox_clr_pts = []
    
    # these are the bytes received from calling the pointcloud
    arr = None
    while True:
        bytes += stream.read(2**14)
        a = bytes.find(b'\xff\xd8\xf1')
        b = bytes.find(b'\xff\xd9\xf2')
        if a != -1 and b != -1:
            data = bytes[a+3:b]
            bytes = bytes[b+3:]
            arr = np.frombuffer(data, dtype=np.float32)
            arr = np.reshape(arr, (76800,3))  
            break


    #we use meshcat animation in order to read and plot data in realtime
    anim = Animation()
    c=0
    with anim.at_frame(vis, 0) as frame:
        c=c+1 #counter that sets an infinite animation

    with anim.at_frame(vis, c) as frame:
        #execute the animation
        vis.set_animation(anim)
        verts= arr.T

    #transposed point cloud
    pointclouds= verts.T
    
    
    pointclouds = pointclouds.reshape(240,320,3)
    clouds_cut = pointclouds[y1:y2,x1:x2,:]
    clouds_cut = np.reshape(clouds_cut, (pix_res,3))
    
    X_ROT = RigidTransform()
    X_ROT.set_translation(np.array([0.,0.,.5]))
    X_ROT.set_rotation(RollPitchYaw(-np.pi/1.5,0,0).ToRotationMatrix())
    

    

    #Filter the points corresponding to the cheezeit box in the point cloud. 
    for i,p in enumerate(pix):
        if p > 0. :                 
            bbox_pos_pts.append(clouds_cut[i])
        else:
            bbox_clr_pts.append(clouds_cut[i])
        ###########################################################
        pass

    
    

    bbox_pos_pts = np.array(bbox_pos_pts) #context points
    bbox_clr_pts = np.array(bbox_clr_pts) #detected object points
    
    
    if len(bbox_pos_pts) > 0:
        new_Point_cloud = X_ROT.multiply(bbox_pos_pts.T).T
        new_All_Points = X_ROT.multiply(verts).T
    else:
        continue

#     #converts list to tuple
#     def convert(list): 
#         return tuple(list) 

#     ##OBB array
#     bbpoints=[] #list for converted points
#     ###
#     #             bbpoints= bbox_pos_pts[1:]
#     obb = OBB.build_from_points(new_Point_cloud)
#     vis["sphere"].set_object(g.Sphere(.001), 
#                              g.MeshLambertMaterial(
#                                  color=0xc9eb34,
#                                  reflectivity=0.8))
#     ##render the center of mass 
#     vis["sphere"].set_transform(
#         tf.translation_matrix(obb.centroid))

#     ##Render Bounding Box
#     #bbox corners
#     c=0
#     bbpoints= obb.points
#     for pt in bbpoints:
#         vis["sph"+str(c)].set_object(g.Sphere(.01), 
#                              g.MeshLambertMaterial(
#                                  color=0xff0000,
#                                  reflectivity=0.8))
#         vis["sph"+str(c)].set_transform(
#         tf.translation_matrix(pt))
#         c+=1

#     #z_lines
#     vertices1= np.transpose(np.array(bbpoints).astype(np.float32))
#     vis['lines_segments1'].set_object(g.LineSegments(g.PointsGeometry(vertices1),g.MeshToonMaterial(color=0xff0000)))

#     #ylines
#     vertices2= np.transpose(np.array([bbpoints[1], bbpoints[2],bbpoints[0], bbpoints[3],bbpoints[5], bbpoints[6],bbpoints[4], bbpoints[7]]).astype(np.float32))
#     vis['lines_segments2'].set_object(g.LineSegments(g.PointsGeometry(vertices2),g.MeshToonMaterial(color=0xff0000)))

#     #xlines
#     vertices3= np.transpose(np.array([bbpoints[0], bbpoints[5],bbpoints[1], bbpoints[4],bbpoints[3], bbpoints[6],bbpoints[2], bbpoints[7]]).astype(np.float32))
#     vis['lines_segments3'].set_object(g.LineSegments(g.PointsGeometry(vertices3),g.MeshToonMaterial(color=0xff0000)))
#     ##########################
  

    with anim.at_frame(vis, 0) as frame:
        c=c+1 #counter that sets an infinite animation
        with anim.at_frame(vis, c) as frame:
            #execute the animation
            vis.set_animation(anim)

            #plot the points

            vis['notfiltered'].set_object(g.PointCloud(position= new_All_Points.T, color=make_meshcat_color_array(new_All_Points.shape[0], 1,1,0),size= 0.001))
            vis['filtered'].set_object(g.PointCloud(position= new_Point_cloud.T, color=make_meshcat_color_array(new_Point_cloud.shape[0], 1,0,0), size= 0.006))
                          
#     
    #print(time.time() - time_start)
    time_start = time.time()



You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


KeyboardInterrupt: 